In [1]:
from lib.eugene import train, test
import torchvision.models as models

from numpy.random import RandomState
import numpy as np
import torch
import torch.optim as optim
from torch.utils.data import Subset
import torch.nn as nn
import torch.nn.functional as F

  
from torchvision import datasets, transforms

In [2]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])

# We resize images to allow using imagenet pre-trained models, is there a better way?
resize = transforms.Resize(224) 

transform_val = transforms.Compose([resize, transforms.ToTensor(), normalize]) #careful to keep this one same
transform_train = transforms.Compose([resize, transforms.ToTensor(), normalize]) 

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device) # you will really need gpu's for this part
    
##### Cifar Data
cifar_data = datasets.CIFAR10(root='.',train=True, transform=transform_train, download=True)
    
#We need two copies of this due to weird dataset api 
cifar_data_val = datasets.CIFAR10(root='.',train=True, transform=transform_val, download=True)
    
# Extract a subset of 100 (class balanced) samples per class

accs = []

for seed in [1,2]:
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 5000))
    indx_train = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[0:10]] for classe in range(0, 10)])
    indx_val = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[10:210]] for classe in range(0, 10)])

    train_data = Subset(cifar_data, indx_train)
    val_data = Subset(cifar_data_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data,
                                             batch_size=128, 
                                             shuffle=True)

    val_loader = torch.utils.data.DataLoader(val_data,
                                           batch_size=128, 
                                           shuffle=False)
    
    #     resnet18 = models.resnet18()
    #     alexnet = models.alexnet()
    #     vgg16 = models.vgg16()
    #     squeezenet = models.squeezenet1_0()
    #     densenet = models.densenet161()
    #     inception = models.inception_v3()
    #     googlenet = models.googlenet()
    #     shufflenet = models.shufflenet_v2_x1_0()
    #     mobilenet = models.mobilenet_v2()
    #     resnext50_32x4d = models.resnext50_32x4d()
    #     wide_resnet50_2 = models.wide_resnet50_2()
    #     mnasnet = models.mnasnet1_0()


    model = models.googlenet(pretrained=True)
    model.classifier = nn.Linear(256 * 6 * 6, 10)
    optimizer = torch.optim.SGD(model.classifier.parameters(), 
                              lr=0.01, momentum=0.9,
                              weight_decay=0.0005)
    model.to(device)
    for epoch in range(10):
        train(model, device, train_loader, optimizer, epoch, display=True)

    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over 2 instances: %.2f +- %.2f'%(accs.mean(),accs.std()))




cuda
Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 100 Num Samples For Val 2000


Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /home/benjamin/.cache/torch/hub/checkpoints/googlenet-1378be20.pth



Train Epoch: 0 [0/100 (0%)]	Loss: 8.146880
Train Epoch: 1 [0/100 (0%)]	Loss: 8.128654
Train Epoch: 2 [0/100 (0%)]	Loss: 8.077737
Train Epoch: 3 [0/100 (0%)]	Loss: 8.146319
Train Epoch: 4 [0/100 (0%)]	Loss: 8.103650
Train Epoch: 5 [0/100 (0%)]	Loss: 8.019337
Train Epoch: 6 [0/100 (0%)]	Loss: 8.140182
Train Epoch: 7 [0/100 (0%)]	Loss: 8.071136
Train Epoch: 8 [0/100 (0%)]	Loss: 8.150529
Train Epoch: 9 [0/100 (0%)]	Loss: 8.094483


/home/benjamin/.local/lib/python3.8/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 8.5938, Accuracy: 1/2000 (0.05%)

Num Samples For Training 100 Num Samples For Val 2000
Train Epoch: 0 [0/100 (0%)]	Loss: 8.151623
Train Epoch: 1 [0/100 (0%)]	Loss: 8.065230
Train Epoch: 2 [0/100 (0%)]	Loss: 8.053954
Train Epoch: 3 [0/100 (0%)]	Loss: 8.148664
Train Epoch: 4 [0/100 (0%)]	Loss: 8.125424
Train Epoch: 5 [0/100 (0%)]	Loss: 8.090104
Train Epoch: 6 [0/100 (0%)]	Loss: 8.026558
Train Epoch: 7 [0/100 (0%)]	Loss: 8.071366
Train Epoch: 8 [0/100 (0%)]	Loss: 8.076777
Train Epoch: 9 [0/100 (0%)]	Loss: 8.050603

Test set: Average loss: 8.5285, Accuracy: 0/2000 (0.00%)

Acc over 2 instances: 0.03 +- 0.03


In [3]:
"Alexnet" Acc over 2 instances: 50.78 +- 0.82

SyntaxError: invalid syntax (<ipython-input-3-6bd317ee73b6>, line 1)